Для правильной работы разместите файлы forecast_chan.csv, market_file.csv, market_money.csv, market_time.csv, money.csv в папку с этим файлом.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# ЗАГРУЗКА ДАННЫХ

In [ ]:
df_market = pd.read_csv('market_file.csv')
df_market

In [ ]:
df_market.info() # описание данных. кроме того видно, что нет пропусков в значениях

In [ ]:
df_market_money = pd.read_csv('market_money.csv')
df_market_money

In [ ]:
df_market_money.info() # описание данных. кроме того видно, что нет пропусков в значениях

In [ ]:
df_market_time = pd.read_csv('market_time.csv')
df_market_time

In [ ]:
df_market_time.info() # описание данных. кроме того видно, что нет пропусков в значениях

In [ ]:
df_money = pd.read_csv('money.csv', sep=';', decimal=',')
df_money

In [ ]:
df_money.info() # описание данных. кроме того видно, что нет пропусков в значениях

In [ ]:
df_forecast_chan = pd.read_csv('forecast_chan.csv', encoding='cp1251', sep=';', decimal=',')
df_forecast_chan

In [ ]:
df_forecast_chan.info() # описание данных. кроме того видно, что нет пропусков в значениях

# ПРЕДОБРАБОТКА + начало исследования

In [ ]:
#market
df_market.head(5)

In [ ]:
for column in df_market.select_dtypes('object').columns:  # описание значений строковых данных
    print(df_market[column].value_counts(), '\n')

In [ ]:
num_cols = df_market.select_dtypes(['int64', 'float64']).columns.drop('id')

# Вычисляем количество столбцов
num_plots = len(num_cols)
n_cols = 1  # Устанавливаем один столбец для всех графиков
n_rows = num_plots  # Каждый график будет находиться в отдельной строке

# Создание объектов рисунков и осей
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(8, 10))

# Построение графиков в столбик
for i, col in enumerate(num_cols):
    sns.boxplot(x=df_market[col], ax=axes[i])

plt.tight_layout()  # Для более компактного отображения
plt.show()


видим вполне адекватные значения, критических выбросов не замечено. Выбросы, которые видны на графике вполне объясняются, в дальнейшем могут пригодится.

In [ ]:
#market money
df_market_money.head(5)

In [ ]:
for column in df_market_money.select_dtypes('object').columns:  # описание значений строковых данных
    print(df_market_money[column].value_counts(), '\n')

In [ ]:
plt.figure(figsize=(15,4))
sns.boxplot(x=df_market_money['Выручка'])

In [ ]:
df_market_money['Выручка'].describe()

In [ ]:
df_market_money.sort_values(by='Выручка', ascending=False).head(5)

видим, что присутствует один человек с неимоверно большой выручкой

давайте рассмотрим, чем вызвана его выручка в текущем месяце. Какая у него активность и т.д.

In [ ]:
df_market[df_market.id == 215380]

In [ ]:
df_market_time[df_market_time.id == 215380] # сразу замечаем опечатку "предыдЦщий_месяц". в дальнейшем надо ее исправить

видим, что человек с этим id НЕ МОГ вызвать такую выручку, т.к. его активность и другие действия такие же, как и у других. все приближено к нормальным значениям. при том время, проведеноое на сайте всего 14 минут. Это ВЫБРОС. удалим человека с этим индексом


In [ ]:
df_market_time = df_market_time[(df_market_time.id != 215380)]
df_market_money = df_market_money[(df_market_money.id != 215380)]
df_money = df_money[(df_money.id != 215380)]
df_market = df_market[(df_market.id != 215380)]
df_forecast_chan = df_forecast_chan[(df_forecast_chan.id != 215380)]

# оставим предыдущий месяц, т.к. там проблем не замечено

In [ ]:
#market time
df_market_time.head(5)


In [ ]:
for column in df_market_time.select_dtypes('object').columns:  # описание значений строковых данных
    print(df_market_time[column].value_counts(), '\n')

In [ ]:
# снова видим отпечатку. заменим ее
df_market_time['Период'] = df_market_time['Период'].apply(lambda x: x if x == 'текущий_месяц' else 'предыдущий_месяц')

In [ ]:
for column in df_market_time.select_dtypes('object').columns:  # описание значений строковых данных
    print(df_market_time[column].value_counts(), '\n')
#  теперь все правильно

In [ ]:
sns.boxplot(df_market_time['минут'])

здесь все хорошо

In [ ]:
#money
df_money.head(5)


In [ ]:
sns.boxplot(df_money['Прибыль'])

все рассмотрено, основные ошибки исправлены

# ПРОДОЛЖАЕМ ИССЛЕДОВАТЕЛЬСКИЙ АНАЛИЗ

In [ ]:
#market 
sns.pairplot(df_market)

In [ ]:
#market time
sns.pairplot(df_market_time)

In [ ]:
sns.histplot(df_market_time['минут'], bins=10)

In [ ]:
#market_money
sns.pairplot(df_market_money)

In [ ]:
sns.histplot(df_market_money['Выручка'], bins=20)

In [ ]:
#market time
sns.pairplot(df_money)

In [ ]:
sns.histplot(df_money['Прибыль'], bins=15)

In [ ]:
df_market

# Объединение таблиц

In [ ]:
merged_data = pd.merge(df_market, df_market_money, on='id')
merged_data = pd.merge(merged_data, df_market_time, on=['id', "Период"])
merged_data

In [ ]:
merged_data.Период.value_counts()

In [ ]:
df_market_money[df_market_money.id == 215348]

In [ ]:
revenue_columns = ['текущий_месяц_выручка', 'предыдущий_месяц_выручка']
revenue_data = merged_data.pivot(index='id', columns='Период', values='Выручка').reset_index()
revenue_data.columns = ['id'] + revenue_columns
time_columns = ['текущий_месяц_время', 'предыдущий_месяц_время']
time_data = merged_data.pivot(index='id', columns='Период', values='минут').reset_index()
time_data.columns = ['id'] + time_columns

# Итоговая таблица с данными о выручке, времени на сайте и остальными данными
final_data = pd.merge(merged_data, revenue_data, on='id')
final_data = pd.merge(final_data, time_data, on='id')

In [ ]:
final_data

In [ ]:
final_data.drop('Период', axis=1, inplace=True)
final_data.drop('Выручка', axis=1, inplace=True)
final_data.drop('минут', axis=1, inplace=True)
final_data

In [ ]:
final_data.drop_duplicates(inplace=True)

In [ ]:
(final_data.id.sort_values() == final_data.id).value_counts()

In [ ]:
money_bef_bef = df_market_money[df_market_money['Период'] == 'препредыдущий_месяц'].drop('Период', axis=1)

In [ ]:
money_bef_bef.columns = ['id', 'препредыдущий_месяц_выручка']
money_bef_bef

In [ ]:
final_data = pd.merge(final_data, money_bef_bef, on='id')

In [ ]:
final_data.columns

In [ ]:
final_data = final_data.reindex(columns=['id', 'Покупательская активность', 'Тип сервиса', 'Разрешить сообщать',
       'Маркет_актив_6_мес', 'Маркет_актив_тек_мес', 'Длительность',
       'Акционные_покупки', 'Популярная_категория',
       'Средний_просмотр_категорий_за_визит',
       'Неоплаченные_продукты_штук_квартал', 'Ошибка_сервиса',
       'Страниц_за_визит', 'текущий_месяц_выручка', 'предыдущий_месяц_выручка','препредыдущий_месяц_выручка', 
       'текущий_месяц_время', 'предыдущий_месяц_время'])


In [ ]:
final_data

# Корреляционный анализ

In [ ]:
from scipy.stats import pearsonr, spearmanr, shapiro

# Выбор только количественных признаков (непрерывных переменных)
numerical_features = final_data.select_dtypes(include=['float64', 'int64'])

# Вычисление корреляций между количественными признаками
correlation_matrix = pd.DataFrame(index=numerical_features.columns, columns=numerical_features.columns)
for i in numerical_features.columns:
    for j in numerical_features.columns:
        if i == j:
            correlation_matrix.loc[i, j] = 1.0
        else:
            _, p_value_i = shapiro(numerical_features[i])
            _, p_value_j = shapiro(numerical_features[j])
            if p_value_i > 0.05 and p_value_j > 0.05:  # Проверка на нормальное распределение
                correlation, _ = pearsonr(numerical_features[i], numerical_features[j])
            else:
                correlation, _ = spearmanr(numerical_features[i], numerical_features[j])
            correlation_matrix.loc[i, j] = correlation

# Вывод матрицы корреляций
correlation_matrix = correlation_matrix.astype('float64')
correlation_matrix

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(correlation_matrix.round(2), annot=True)

# Сегментация

было решено рассмотреть людей, со среднемесячной прибылью больше среднего, которые потенциально могут понизить покупательскую активность.

In [ ]:
# объеденим таблицы с прибылью, рещультатом моделирования и оставшуюся, над которой мы вели работу
df_for_segment = pd.merge(final_data, df_money, on='id')
df_for_segment = pd.merge(df_for_segment, df_forecast_chan, on='id')

In [ ]:
df_for_segment.head(5)

In [ ]:
sns.boxplot(df_for_segment.Прибыль) # смотрим на среднюю и медиану для отбора сегмента

In [ ]:
df_for_segment.Прибыль.mean(), df_for_segment.Прибыль.median() # для простоты округлим до 4

In [ ]:
sns.boxplot(df_for_segment.вероятность)

In [ ]:
df_for_segment.вероятность.mean(), df_for_segment.вероятность.median()
# видим, что примерно половина значений лежит в диапазоне [0, 0.12]. Возьмем для сегментации значения [0.3,1]

In [ ]:
# отбор значений
df_for_segment = df_for_segment[(df_for_segment['Прибыль'] >= 4) & (df_for_segment['вероятность'] >= 0.3)]

0.31    2 0.4  449

In [ ]:
df_for_segment.head(5)

отобрав значения, сразу же

In [ ]:
# Выбор только количественных признаков (непрерывных переменных)
numerical_features = df_for_segment.select_dtypes(include=['float64', 'int64'])

# Вычисление корреляций между количественными признаками
correlation_matrix = pd.DataFrame(index=numerical_features.columns, columns=numerical_features.columns)
for i in numerical_features.columns:
    for j in numerical_features.columns:
        if i == j:
            correlation_matrix.loc[i, j] = 1.0
        else:
            _, p_value_i = shapiro(numerical_features[i])
            _, p_value_j = shapiro(numerical_features[j])
            if p_value_i > 0.05 and p_value_j > 0.05:  # Проверка на нормальное распределение
                correlation, _ = pearsonr(numerical_features[i], numerical_features[j])
            else:
                correlation, _ = spearmanr(numerical_features[i], numerical_features[j])
            correlation_matrix.loc[i, j] = correlation

# Вывод матрицы корреляций
correlation_matrix = correlation_matrix.astype('float64')
correlation_matrix
plt.figure(figsize=(10, 10))
sns.heatmap(correlation_matrix.round(2), annot=True)

видим отрицательную корреляцию Маркет_актив_6_мес и вероятность.
а также Маркет_актив_6_мес слабо коррелирует с количеством страниц за визит, а также со временем текущего и предыдущего месяцев

Рассмотрим эти взаимосвязи

In [ ]:
plt.figure(figsize=(5, 15))  

plt.subplot(4, 1, 1)  
sns.scatterplot(x=df_for_segment['Маркет_актив_6_мес'], y=df_for_segment['вероятность']) 

# График 2
plt.subplot(4, 1, 2)  
sns.scatterplot(x=df_for_segment['Маркет_актив_6_мес'], y=df_for_segment['Страниц_за_визит'])  

# График 3
plt.subplot(4, 1, 3)  
sns.scatterplot(x=df_for_segment['Маркет_актив_6_мес'], y=df_for_segment['текущий_месяц_время'])  

plt.subplot(4, 1, 4)  
sns.scatterplot(x=df_for_segment['Маркет_актив_6_мес'], y=df_for_segment['предыдущий_месяц_время'])
plt.show()  # Показать все три графика


видим небольшие закономерности. Построим графики линейных регрессий

In [ ]:
sns.lmplot(data = df_for_segment, x='Маркет_актив_6_мес', y='вероятность')  


In [ ]:
sns.lmplot(data = df_for_segment, x='Маркет_актив_6_мес', y='Страниц_за_визит')  


In [ ]:
sns.lmplot(data = df_for_segment, x='Маркет_актив_6_мес', y='текущий_месяц_время')  


In [ ]:
sns.lmplot(data = df_for_segment, x='Маркет_актив_6_мес', y='предыдущий_месяц_время')  


видим большие отклонения значений от результата линейной регрессии везде, кроме графика Маркет_актив_6_мес и вероятность. В этом графике отклонения не столь большие.

итак, действительно, чем больше среднемесячное значение маркетинговых коммуникаций компании (Маркет_фктив_6_мес), которое приходилось на покупателя за последние 6 месяцев, тем менее вероятно снижение активности покупателя, и тем больше время проведенное на сайте за предыдущий и текущий месяцы.
давайе рассмотрим также взаимосвязь этой активности с временем за два последних месяца(время за предыдущий + время за текущий месяц)

In [ ]:
df_for_segment['время_последние_месяцы'] = df_for_segment['текущий_месяц_время'] + df_for_segment['предыдущий_месяц_время']

In [ ]:
shapiro(df_for_segment.время_последние_месяцы)

In [ ]:
df_for_segment.время_последние_месяцы.corr(df_for_segment['Маркет_актив_6_мес'])

In [ ]:
# видим, что корреляция стала больше.
sns.lmplot(data=df_for_segment, x='Маркет_актив_6_мес', y = 'время_последние_месяцы')

однако отклонений меньше не стало.

чем больше среднемесячное значение маркетинговых коммуникаций компании (Маркет_фктив_6_мес), которое приходилось на покупателя за последние 6 месяцев, тем менее вероятно снижение активности покупателя, и тем больше время проведенное на сайте за последние два месяца.
следует продвигать активность маркета для этого сегмента.

# Микровывод
итак, был рассмотрен сегмент людей, со среднемесячной прибылью больше среднего, которые потенциально могут понизить покупательскую активность.

было выяснено, что активность маркета положительно влияет на время проведенное на сайте, также уменьшает вероятность снижения активности покупателя. Ее следует продвигать для этого сегмента людей. Для этого сегмента стоит сделать упор на рекламу, рассылки, звонки и т.д.

# Вывод

задача: разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.



данные

были даны пять таблиц: 
основная таблица, описывающая клиента, коммуникацию с ним, продуктовое поведение, финаносовое поведение, а также поведение на сайте клиента

таблица с выручкой, получакемой магазином от покупателя в разные периоды

таблица со временем, проведенным на сайте в разные периоды

таблица с прибылью получаемой магазином от продаж каждому покупателю

таблица с работой специалиста Data Science с вероятность. снижения покупательской активности клиента.


были удалены выбросы, связанные с выручкой клиенты, исправлены опечатки в таблицах

Итог:

было выявлено, что для сегмента людей, со среднемесячной прибылью больше среднего, которые потенциально могут понизить покупательскую активность для ее повышения, следует увеличивать рекламу, рассылки, звонки. Стоит рассмотреть возможность взаимодействия с клиентом путем социальных сетей, становящихся все более популярными, и других путей коммуникации.